In [2]:
using DataFrames
using Chemfiles

home = "/home/german/labo/18/egfr/model/"

"/home/german/labo/18/egfr/model/"

In [20]:
function res_3_to_1(in_3::AbstractString)
    
    res = lowercase(in_3)
    out_1 = AbstractString
    
    if res == "ala"
        out_1 = "A"
    elseif res == "arg"
        out_1 = "R"
    elseif res == "asn"
        out_1 = "N"
    elseif res == "asp"
        out_1 = "D"
    elseif res == "cys"
        out_1 = "C"
    elseif res == "gln"
        out_1 = "Q"
    elseif res == "glu"
        out_1 = "E"
    elseif res == "gly"
        out_1 = "G"
    elseif res == "his"
        out_1 = "H"
    elseif res == "ile"
        out_1 = "I"
    elseif res == "leu"
        out_1 = "L"
    elseif res == "lys"
        out_1 = "K"
    elseif res == "met"
        out_1 = "M"
    elseif res == "phe"
        out_1 = "F"
    elseif res == "pro"
        out_1 = "P"
    elseif res == "ser"
        out_1 = "S"
    elseif res == "thr"
        out_1 = "T"
    elseif res == "trp"
        out_1 = "W"
    elseif res == "tyr"
        out_1 = "Y"
    elseif res == "val"
        out_1 = "V"
    else
        error("Unknown aminoacid: ", in_3)
    end
    
    return out_1
end

function res_3_to_1(in_vec::Array{String, 1})

    out_vec = Array{String, 1}(length(in_vec))
    i = 0
    for each in in_vec
        i+=1
        try 
            out_vec[i] = res_3_to_1(each)
        catch e
            error("Malformed array list.\nCaught error: $e")
        end
    end
    
    return out_vec
end

res_3_to_1 (generic function with 2 methods)

In [21]:
function seq_from_top(in_top::Chemfiles.Topology)
    # Get 1 letter code amino acid list from a topology
    aa = convert(Int64, count_residues(in_top))
    ids = Array{Int64, 1}(aa)

    for i=1:aa
        ids[i] = id(Residue(in_top, i-1))
    end
    ids_sorted_indices = sortperm(ids);

    seq_3 = Array{String, 1}(aa)
    i = 0
    for each in ids_sorted_indices
        i+=1
        seq_3[i] = name((Residue(in_top, each-1)))
    end
    seq_1 = res_3_to_1(seq_3);
    
    return seq_1
end

seq_from_top (generic function with 1 method)

In [4]:
tmp = readdlm(string(home, "/pdbs.list"))
pdbs_list = convert(Array{String, 1}, reshape(tmp, length(tmp)));

# Leo alineamiento

In [9]:
alineamientos = Array{Array{Any, 2}, 1}(length(pdbs_list))
indices_mut = Array{Array{Int64, 1}, 1}(length(pdbs_list))
indices_mis = Array{Array{Int64, 1}, 1}(length(pdbs_list))

j = 0

for each in pdbs_list
    j += 1
    println(each) 
    
    in_seq_canonical = Array{AbstractString, 1}
    in_seq_each = Array{AbstractString, 1}
    in_mut_each = Array{AbstractString, 1}

    seq_canonical = Array{AbstractString, 1}
    seq_each = Array{AbstractString, 1}
    mut_each = Array{AbstractString, 1}
    
    
    # Leo archivo.
    in_filename = string(home, each, "/alignment_", each)
    alignment_file = open(in_filename)
    lineas = readlines(alignment_file)

    k = 0
    # Ahora leo todas las líneas y tomo los arrays en bruto
    for linea in lineas
        k+=1
        inicio = 31
        final = length(lineas) - 3
    
        if inicio < k < final
            n_linea = k - 31
            altura = ((n_linea - 1 + 4) % 4)
        
            if altura == 0
                in_seq_canonical = [ in_seq_canonical ; linea[22:end-7] ]
            elseif altura == 1
                in_mut_each = [ in_mut_each ; linea[22:end] ]
            elseif altura == 2
                in_seq_each = [ in_seq_each ; linea[22:end-7] ]
            end
        end
    end
    
    # Paso las secuencias en bruto a arrays bien formados.
    for i = 2:length(in_seq_canonical)
        tmp_canonical = [ in_seq_canonical[i][j] for j = 1:length(in_seq_canonical[i]) ]
        seq_canonical = [ seq_canonical ; tmp_canonical]
    
        tmp_each = [ in_seq_each[i][j] for j = 1:length(in_seq_each[i]) ]
        seq_each = [ seq_each ; tmp_each]

        tmp = [ in_mut_each[i][j] for j = 1:length(in_mut_each[i]) ]
        mut_each = [ mut_each ; tmp]
    end

    # Determino q pdbs tienen mutaciones y en q posiciones.
    tmp_indices_mut = findin(mut_each, ".")
    indices_mut[j] = tmp_indices_mut
    
    # Determino las regiones a modelar
    tmp_indices_mis = findin(mut_each, " ") .- 1
    indices_mis[j] = tmp_indices_mis

    # Guardo todo
    alineamiento = [ seq_canonical seq_each mut_each ]
    alineamientos[j] = alineamiento 
end

1M14_A
1M17_A
1XKK_A
2EB2_A
2GS2_A
2GS6_A
2GS7_A
2ITN_A
2ITP_A
2ITU_A
2ITX_A
2ITZ_A
2J5F_A
2RGP_A
3BEL_A
3GOP_A
3GT8_A
3GT8_C
3IKA_A
3IKA_B
3LZB_A
3UG1_A
3VJN_A
3W2O_A
3W2R_A
3W2S_A
3W32_A
3W33_A
4G5J_A
4I1Z_A
4I22_A
4I23_A
4I24_A
4LI5_A
4LQM_A
4R3P_A
4R5S_A
4RJ4_A
4ZAU_A
4ZJV_A
5C8K_A
5CAO_A
5CAP_A
5CAV_A
5CNN_A
5CNN_B
5CNO_A
5CZH_A
5EDP_A
5EM5_A
5EM6_A
5EM7_A
5EM8_A
5HG5_A
5HG7_A
5HG8_A
5HIB_A
5HIC_A


### Renumero los indices missings. La numeración original es según
### las posiciones del alineamiento. Quiero obtener la numeración según
### la sequencia canonical

In [12]:
mod_indices_mis = Array{Array{Int64, 1}, 1}(length(pdbs_list))

for i = 1:length(pdbs_list)
    tmp_indices = Array{Int64}[]
    
    # La secuencia a modelar empieza y termina con leucina
    pre_mask = findin(alineamientos[i][:, 1], "L")
    mask = pre_mask[1] .< indices_mis[i] .< pre_mask[end]
    mod_indices_mis[i] = indices_mis[i][mask] .- pre_mask[1] .+ 2
    # Ya tengo los índices de missings numerados según la seq canonical.
end

In [17]:
indices_mis[3]

44-element Array{Int64,1}:
   1
  33
  34
  35
  36
  49
  50
  51
  52
 166
 167
 168
 169
   ⋮
 294
 295
 296
 297
 298
 299
 300
 301
 302
 303
 304
 305

In [18]:
mod_indices_mis[3]

16-element Array{Int64,1}:
  32
  33
  34
  35
  48
  49
  50
  51
 165
 166
 167
 168
 169
 170
 171
 172

In [22]:
i = 3
each = pdbs_list[i]
in_pdb_trj = Trajectory(string(home, each, "/", each, ".pdb"))
in_pdb_frm = read(in_pdb_trj)
in_pdb_top = Topology(in_pdb_frm)
seq = seq_from_top(in_pdb_top)

289-element Array{String,1}:
 "A"
 "L"
 "L"
 "R"
 "I"
 "L"
 "K"
 "E"
 "T"
 "E"
 "F"
 "K"
 "K"
 ⋮  
 "M"
 "D"
 "E"
 "V"
 "V"
 "D"
 "A"
 "D"
 "E"
 "Y"
 "L"
 "I"

In [29]:
in_pdb_top.handle

Ptr{Chemfiles.lib.CHFL_TOPOLOGY} @0x000000000581f1a0

In [25]:
(Residue(in_pdb_top, 0))

Chemfiles.Residue(Ptr{Chemfiles.lib.CHFL_RESIDUE} @0x0000000001caeaf0)